In [1]:
from sage.matrix.berlekamp_massey import berlekamp_massey
from random import sample
import time


# Parameters for 128-bit security
n = 17669     # Ring size
n1 = 46       # Reed-Solomon dimension
n2 = 384      # Reed-Muller dimension
n1n2 = 17664  # n1*n2
l = 5         # n-n1n2
mult = 3      # Multiplier for Reed-Muller encoding
k = 16        # Reed-Solomon code dimension
w = 66        # Secret vector weight
we = 75       # Error weight (w_e=w_r)


# Reed-Muller
RM_C = codes.ReedMullerCode(GF(2), 1, 7)
RM_G = RM_C.generator_matrix()
H = hadamard_matrix(128)

# Base ring over GF(2)
R.<x> = GF(2)[]
P = x^n + 1
Rq = R.quotient(P, 'xbar')
xbar = Rq.gen()

# Reed-Solomon
Rz.<z> = PolynomialRing(GF(2))
F.<a> = GF(2^8, name='a', modulus=z^8 + z^4 + z^3 + z^2 + 1)
delta = (n1 - k) // 2
PR.<X> = PolynomialRing(F)
g = prod([X - a^i for i in range(1, n1 - k + 1)])


# ------------------ UTILITIES ------------------ #

# Generate a random vector of size n and weight w
def random_sparse_vector(n, w):
    positions = sample(range(n), w)
    vec = [1 if i in positions else 0 for i in range(n)]
    return vector(GF(2), vec)

# Convert a vector into a polynomial in the ring Rq
def vector_to_poly(v):
    return Rq(v.list())

# Convert a polynomial into a list
def poly_to_vector(p, n):
    return [int(p[i]) for i in range(n)]

# Truncate the most significant l bits
def truncate(poly, l):
    return poly.mod(x^ (n - l))

# Convert GF(2^8) element to 8-bit vector over GF(2)
def byte_to_bits(elem):
    coeffs = elem.polynomial().coefficients(sparse=False)
    return vector(GF(2), coeffs + [0] * (8 - len(coeffs)))  # pad to 8 bits

# Convert 8-bit vector over GF(2) to GF(2^8) element
def bits_to_byte(bits):
    assert len(bits) == 8, "Input must be length-8 vector"
    poly = sum([bits[i] * z^i for i in range(8)])
    return F(poly)


# ------------------ REED-MULLER ------------------ #

# Encodes a word into a Reed-Muller codeword
def ReedMullerEncode(m):
    encoded = (m * RM_G).list() * mult
    return vector(GF(2), encoded)

# Decodes a Reed-Muller codeword into a word
def ReedMullerDecode(encoded_m):
    chunks = [encoded_m[i::128] for i in range(128)]
    Fv = vector(ZZ, [sum((-1)**b for b in chunk) for chunk in chunks])
    HF = H * Fv
    maxval = max([abs(x) for x in HF])
    pos = [i for i in range(len(HF)) if abs(HF[i]) == maxval][0]
    x_bin = Integer(pos).digits(base=2, padto=7)
    if HF[pos] > 0:
        decoded = vector(GF(2), [0] + x_bin)  
    else:
        decoded = vector(GF(2), [1] + x_bin)
    return decoded
    

# ------------------ REED-SOLOMON ------------------ #

# Encodes a word into a Reed-Solomon codeword
def ReedSolomonEncode(msg_bytes):
    msg_poly = PR(msg_bytes) * X^(n1 - k)
    parity = (msg_poly % g).list()
    return parity + msg_bytes

# Compute the syndromes
def compute_syndromes(received):
    r_poly = PR(received)
    return [r_poly(a^i) for i in range(1, 2 * delta + 1)]

# Finds the positions of the errors
def find_error_positions(sigma):
    return [j for j in range(n1) if sigma(a^(-j)) == 0]

# Computes the polynomial Z
def compute_Z(S, sigma):
    S_poly = PR(S) 
    return (S_poly * sigma) % X**(2 * delta)

# Compute the error values
def compute_error_values(Z, sigma, positions):
    sigma_prime = sigma.derivative()
    e = [F(0)] * n1
    for j in positions:
        x_inv = a^(-j)
        e[j] = Z(x_inv) / sigma_prime(x_inv)
    return e

# Corrects the errors
def correct_errors(received, errors):
    return [received[i] + errors[i] for i in range(n1)]

# Decodes a Reed-Muller codeword into a word
def ReedSolomonDecode(received):
    S = compute_syndromes(received)
    if all(s == 0 for s in S):
        return received[-k:]       
    sigma = PR(berlekamp_massey(S).reverse())
    positions = find_error_positions(sigma)
    Z = compute_Z(S, sigma)
    errors = compute_error_values(Z, sigma, positions)
    corrected = correct_errors(received, errors)
    return corrected[-k:]
    

# ------------------ EN/DE-CODE -------------------- #

# Encodes a message in GF(2^8) with length 16 using concatenated codes, outputtong a polynimial in the ring Rq
# Uses Reed-Solomon as the internal code and Reed-Muller as the external one
def encode(msg):
    assert len(msg) == 16
    rs_codeword = ReedSolomonEncode([F(b) for b in msg])

    # Convert field elements to 8-bit binary vectors
    rm_bits = [byte_to_bits(byte) for byte in rs_codeword]

    # Reed-Muller encode each byte separately
    rm_encoded_chunks = [ReedMullerEncode(bits) for bits in rm_bits]

    # Concatenate all RM-encoded chunks into a single vector
    rm_encoded = vector(GF(2), sum([list(v) for v in rm_encoded_chunks], []))
    return vector_to_poly(rm_encoded)

# Decodes a polynomial in Rq into a message in GF(2^8), using the same concatenated codes
def decode(m):
    m_vector = poly_to_vector(m, n1n2)
    # Break m_vector into chunks corresponding to Reed-Muller codewords
    rm_chunks = [m_vector[i:i+n2] for i in range(0, n1n2, n2)]

    # RM decode each chunk back to 8 bits
    rs_bits = [ReedMullerDecode(chunk) for chunk in rm_chunks]
    rs_symbols = [bits_to_byte(bits) for bits in rs_bits]
        
    # RS decode
    decoded_msg = ReedSolomonDecode(rs_symbols)
    return decoded_msg


# ------------------ HQC FUNCTIONS ------------------ #

# Generates the public and private keys
def hqc_keygen():
    h = Rq.random_element()
    x = vector_to_poly(random_sparse_vector(n, w))
    y = vector_to_poly(random_sparse_vector(n, w))
    s = x + h * y
    return (h, s), (x, y)

# Encrypts a message (in 16*GF(2^8)) using the public keys
def hqc_encrypt(h, s, msg):
    m = encode(msg)

    r1 = vector_to_poly(random_sparse_vector(n, we))
    r2 = vector_to_poly(random_sparse_vector(n, we))
    e  = vector_to_poly(random_sparse_vector(n, we))

    u = r1 + h * r2
    v = m + s * r2 + e
    return u, truncate(v, l)

# Decrypts the original message from the ciphertexts using the secret key y
def hqc_decrypt(u, v, y):
    m = v + u * y
    return decode(m)




# ------------------ EXAMPLE USAGE ------------------ #
msg = [F.random_element() for _ in range(k)]
start = time.time()
(pk, sk) = hqc_keygen()
keygen_time = time.time()
u, v = hqc_encrypt(pk[0], pk[1], msg)
encrypt_time = time.time()
recovered = hqc_decrypt(u, v, sk[1])
decrypt_time = time.time()

print("Original:", msg)
print("Recovered:", recovered)
print("Correct:", msg == recovered)
print("Keygen time:   {:.4f} s".format(keygen_time - start))
print("Encrypt time:  {:.4f} s".format(encrypt_time - keygen_time))
print("Decrypt time:  {:.4f} s".format(decrypt_time - encrypt_time))
print("Total time:  {:.4f} s".format(decrypt_time - start))

(h, s) = pk

Original: [a^7 + a^5 + a^4 + a^3 + a^2 + 1, a^6 + a^5 + a^4 + a^3 + 1, a^7 + a^5 + a^3 + a^2 + a + 1, a^7 + a^6 + a^5 + a^3 + a^2 + 1, a^7 + a^6 + a^4 + a^3 + a^2, a^7 + a^6 + a^5 + 1, a^7 + a^6 + a^5 + a^3 + a, a^7 + a^3 + a^2 + a + 1, a^7 + a^6 + a^5 + a^4 + 1, a^7 + a^4 + a^3 + a^2 + a + 1, a^7 + a^5 + a^3 + 1, a^7 + a^5 + a^3 + a^2 + a, a^7 + a^6 + a^5 + a^4 + a, a^7 + a^4 + a^3 + a^2 + a + 1, a^7 + a^6 + a^4 + a^2 + a, a^5 + a^3 + a^2 + a + 1]
Recovered: [a^7 + a^5 + a^4 + a^3 + a^2 + 1, a^6 + a^5 + a^4 + a^3 + 1, a^7 + a^5 + a^3 + a^2 + a + 1, a^7 + a^6 + a^5 + a^3 + a^2 + 1, a^7 + a^6 + a^4 + a^3 + a^2, a^7 + a^6 + a^5 + 1, a^7 + a^6 + a^5 + a^3 + a, a^7 + a^3 + a^2 + a + 1, a^7 + a^6 + a^5 + a^4 + 1, a^7 + a^4 + a^3 + a^2 + a + 1, a^7 + a^5 + a^3 + 1, a^7 + a^5 + a^3 + a^2 + a, a^7 + a^6 + a^5 + a^4 + a, a^7 + a^4 + a^3 + a^2 + a + 1, a^7 + a^6 + a^4 + a^2 + a, a^5 + a^3 + a^2 + a + 1]
Correct: True
Keygen time:   0.5812 s
Encrypt time:  0.6280 s
Decrypt time:  0.5292 s
Total t

In [2]:
# Returns the weight of a polynomial
def weight(poly):
    return sum(int(c) for c in poly.list())

# This is the polynomial (in Rq) with every coefficient 1
ones = Rq([1] * n)

# Solve s = h * y even if h is not invertible, knowing (the parity of) w the weight of y
def linsolve(h,s,w=w):
    if weight(h)%2==1:
        y = h.inverse()*s
    else:
        if w%2==0:
            y = (h+ones).inverse()*s
        else:
            y = (h+ones).inverse()*s+ones
    return y

In [3]:
# Crackable if the secret key is reused for a different public key

# set up
h1 = Rq.random_element()
s1 = sk[0] + h1 * sk[1]

# attack
y = linsolve(h+h1,s+s1)

print('Obtained the secret key:', y==sk[1])

Obtained the secret key: True


In [4]:
# set up to utilize the encryption one time use vectors (which shouldn't happen)
m = encode(msg)

r1 = vector_to_poly(random_sparse_vector(n, we))
r2 = vector_to_poly(random_sparse_vector(n, we))
e  = vector_to_poly(random_sparse_vector(n, we))

u = r1 + h * r2
v = m + s * r2 + e
v = truncate(v,l)

In [5]:
# Crackacble if the r's parameters are reused with a different public key

# set up
u1 = r1 + h1 * r2

# attack
at_r2 = linsolve(h+h1,u+u1,we)

print('Obtained r2:', at_r2==r2)
print('Obtained the message:', decode(v-s*at_r2)==msg) #If you know r2 then you can decode the message without the secret key

Obtained r2: True
Obtained the message: True


In [6]:
# Crackable if r2 is reused to encrypt more than one message, and the attacker knows one of them (and uses the same public key)

# set up
msg1 = [F.random_element() for _ in range(k)]
m1 = encode(msg1)
e1  = vector_to_poly(random_sparse_vector(n, we))
v1 = truncate(m1 + s * r2 + e1, l)

# attack
dmsg = decode(v+v1)
at_msg = [a + b for a, b in zip(dmsg, msg1)]

print('Obtained the original message:', at_msg==msg)

Obtained the original message: True


In [7]:
# Computes the intersection of two polynomials, that is, returns the common non-zero coefficients
def intersection(poly1, poly2):
    coeffs1 = poly1.list()
    coeffs2 = poly2.list()
    result_coeffs = [a and b for a, b in zip(coeffs1, coeffs2)]
    return Rq(result_coeffs)

# Receives a vector r2 with weight we+1 and corrects it, erasing the index of the extra coefficient, using public key s and ciphertext v
def correct_i(r2, s, v):
    support = [i for i, c in enumerate(r2.list()) if c == 1]

    for i in support:
        # Build monomial x^i in Rq
        monomial = Rq([0]*i + [1])
        
        # Remove x^i from r2 by subtracting it
        new_r2 = r2 + monomial  # Since GF(2), this flips the bit
        test = v + s * new_r2
        
        if weight(encode(decode(test)) + test) == we:
            return new_r2
    
    return None

# Receives a vector r2 with weight we+2 and corrects it, erasing the indeces of the 2 extra coefficients, using public key s and ciphertext v
def correct_two_positions(r2, s, v):
    support = [i for i, c in enumerate(r2.list()) if c == 1]
    
    for idx1 in range(len(support)):
        for idx2 in range(idx1 + 1, len(support)):
            i, j = support[idx1], support[idx2]

            # Create x^i + x^j in Rq
            diff_poly = Rq([0]*i + [1]) + Rq([0]*j + [1])
            new_r2 = r2 + diff_poly  # flip both i and j bits
            test = v + s * new_r2

            if weight(encode(decode(test)) + test) == we:
                return new_r2
    
    return None

# Receives two vectors r2 and r2_1 with a switched coefficient between them and corrects r2, using public key s and ciphertext v
def find_two_positions_2(r2, r2_1, s, v):
    support = [i for i, c in enumerate(r2.list()) if c == 1]
    support_1 = [i for i, c in enumerate(r2_1.list()) if c == 1]
    
    for idx1 in range(len(support)):
        for idx2 in range(len(support_1)):
            i, j = support[idx1], support_1[idx2]

            # Create x^i + x^j in Rq
            diff_poly = Rq([0]*i + [1]) + Rq([0]*j + [1])
            new_r2 = r2 + diff_poly  # flip both i and j bits
            test = v + s * new_r2

            if weight(encode(decode(test)) + test) == we:
                return new_r2
    
    return None

In [8]:
# If encryption uses a fixed e, it is possible to recover the message by encrypting it several times using the same public key

# set up
r2_1 = vector_to_poly(random_sparse_vector(n, we))
r2_2 = vector_to_poly(random_sparse_vector(n, we))
v1 = truncate(m + s * r2_1 + e, l)
v2 = truncate(m + s * r2_2 + e, l)

# attack
r2_01 = linsolve(s, v+v1, 2*we)
r2_02 = linsolve(s, v+v2, 2*we)
r2_12 = linsolve(s, v1+v2, 2*we)
print('Weight of 01:', weight(r2_01))
print('Weight of 02:', weight(r2_02))
print('Weight of 03:', weight(r2_12))
at_r2_0 = intersection(r2_01, r2_02)
at_r2_1 = intersection(r2_01, r2_12)
at_r2_2 = intersection(r2_02, r2_12)
print('Weight of the 1 intersection:', weight(at_r2_0))
print('Weight of the 2 intersection:', weight(at_r2_1))
print('Weight of the 3 intersection:', weight(at_r2_2))
if weight(at_r2_0)==75 and weight(at_r2_1)==75 and weight(at_r2_2)==75:
    print('Obtained r2:', at_r2_0==r2)
elif weight(at_r2_0)==76:
    at_r2 = correct_i(at_r2_0,s,v)
    print('Obtained r2:', at_r2==r2)
elif weight(at_r2_1)==76:
    at_r2 = correct_i(at_r2_1,s,v1)
    print('Obtained r2_1:', at_r2==r2_1)
elif weight(at_r2_2)==76:
    at_r2 = correct_i(at_r2_2,s,v2)
    print('Obtained r2_2:', at_r2==r2_2)
else:
    print('If one of the weights is 77 then you can use find_two_positions(),\
 but it would take (probably) almost an hour (there are 77 choose 2 = 2926 possibilities)')
    print('If two of the weights are 75 and the other is 73, then you can use find_two_positions_2(),\
 but there are 75^2=5625 possibilities, which would take (probably) almost two hours ')
    print('It still is a reasonable amount of time, but a faster way is to get a fourth v, that is, a fourth encryption')

Weight of 01: 150
Weight of 02: 150
Weight of 03: 150
Weight of the 1 intersection: 75
Weight of the 2 intersection: 75
Weight of the 3 intersection: 75
Obtained r2: True


In [9]:
# Faulty decoder that leaks information
# Commented the actual decoding to be faster, since we won't use the decoded message
def f_decode(m):
    m_vector = poly_to_vector(m, n1n2)
    # Break m_vector into chunks corresponding to Reed-Muller codewords
    rm_chunks = [m_vector[i:i+n2] for i in range(0, n1n2, n2)]

    # # RM decode each chunk back to 8 bits
    # rs_bits = [ReedMullerDecode(chunk) for chunk in rm_chunks]
    # rs_symbols = [bits_to_byte(bits) for bits in rs_bits]
        
    # # RS decode
    # decoded_msg = ReedSolomonDecode(rs_symbols)
    return rm_chunks[0] #, decoded_msg

# Decryption machine (you don't need to give it the secret key) that uses the faulty decoder 
def f_decrypt(u, v):
    m = v - u * sk[1]
    return f_decode(m)


In [10]:
# If you have a decryption machine that leaks a chunk, you can find the secret key via chosen ciphertext attack

# attack
y=[]
for i in range(0, n1n2, n2):
    y += f_decrypt(xbar^-i,0)
    
print('Obtained the secret key:', Rq(y)==sk[1])

Obtained the secret key: True


In [11]:
# Given a polynomial and a fraction in (0,1), returns a list with that fraction of the coefficients and None's in the others
def simulate_leakage(poly, leak_fraction):
    coeffs = poly.list()
    num_leaked = int(leak_fraction * n)
    leak_positions = set(sample(range(n), num_leaked))
    
    leaked_coeffs = [coeffs[i] if i in leak_positions else None for i in range(n)]
    return leaked_coeffs

# Given a list with some coefficients and None's, returns a polynomial with the known coefficients and, by default, with the rest as zeros
def fill_unknowns(leaked_coeffs, fill=0):
    filled = [b if b is not None else fill for b in leaked_coeffs]
    return Rq(filled)

# Returns v with a cyclic shift
def cyclic_shift(v, shift):
        shift = shift % n
        return v[-shift:] + v[:-shift]

# Solves the linear system s = x + h * y, (only correspond to the secret keys if the system is determined)
def solve_linear_system(s, h, known_x, known_y):

    # Indices of unknown bits
    unknown_x_indices = [i for i, val in enumerate(known_x) if val is None]
    unknown_y_indices = [i for i, val in enumerate(known_y) if val is None]
    
    # 1. Compute the right-hand side
    rhs = s + fill_unknowns(known_x) + h * fill_unknowns(known_y)
    
    # 2. Build the system matrix
    # For each unknown bit in x, its coefficient is 1 at its position
    # For each unknown bit in y, its coefficient is h at its position  
    
    num_unknowns = len(unknown_x_indices) + len(unknown_y_indices)
    M = Matrix(GF(2), n, num_unknowns)
    
    # Fill in columns for unknown x bits
    for col, idx in enumerate(unknown_x_indices):
        M[idx, col] = 1
    
    
    # Fill in columns for unknown y bits
    for col, idx in enumerate(unknown_y_indices, start=len(unknown_x_indices)):
        h_col = vector(GF(2), h.list())
        M.set_column(col, vector(GF(2), cyclic_shift(list(h_col), idx)))
    
    # 3. Convert rhs to vector
    rhs_vec = vector(GF(2), rhs.list())
    
    # 4. Solve the system
    sol = M.solve_right(rhs_vec)
    
    # 5. Reconstruct full x and y
    x_full = known_x[:]
    y_full = known_y[:]
    for i, idx in enumerate(unknown_x_indices):
        x_full[idx] = sol[i]
    for i, idx in enumerate(unknown_y_indices):
        y_full[idx] = sol[len(unknown_x_indices) + i]

    return Rq(x_full), Rq(y_full)


In [12]:
# If we manage to get half of x and y, we can recover the rest of the secret key

# set up
known_x = simulate_leakage(sk[0],0.5)
known_y = simulate_leakage(sk[1],0.5)

# attack
x,y = solve_linear_system(s, h, known_x, known_y)

print('Obtained the secret key:',y==sk[1])

Obtained the secret key: True


In [13]:
# It doesn't have to be half of each, just half of the total

# set up
known_x = simulate_leakage(sk[0],0.3)
known_y = simulate_leakage(sk[1],0.7)

# attack
x,y = solve_linear_system(s, h, known_x, known_y)

print('Obtained the secret key:',y==sk[1])

Obtained the secret key: True


In [14]:
# It is possible to solve the system even if it is undetermined, but it grows exponentially with each free variable

from itertools import product

def solve_undetermined(h, s, known_x, known_y, max_enumerate, weight=w):

    # Indices of unknown bits
    unknown_x_indices = [i for i, val in enumerate(known_x) if val is None]
    unknown_y_indices = [i for i, val in enumerate(known_y) if val is None]
    
    # Build right-hand side
    x_known_vec = [b if b is not None else 0 for b in known_x]
    y_known_vec = [b if b is not None else 0 for b in known_y]
    rhs = s + fill_unknowns(x_known_vec) + h * fill_unknowns(y_known_vec)
    rhs_vec = vector(GF(2), rhs.list())

    # Build matrix
    num_unknowns = len(unknown_x_indices) + len(unknown_y_indices)
    M = Matrix(GF(2), n, num_unknowns)
    # x unknowns
    for col, idx in enumerate(unknown_x_indices):
        M[idx, col] = 1
    # y unknowns

    for col, idx in enumerate(unknown_y_indices, start=len(unknown_x_indices)):
        h_col = vector(GF(2), h.list())
        M.set_column(col, vector(GF(2), cyclic_shift(list(h_col), idx)))

    # Solve the system
    sol = M.right_kernel().basis()
    particular = M.solve_right(rhs_vec)

    if num_free > max_enumerate:
        print("Too many free variables to enumerate all solutions.")
        return None

    # Enumerate all possible solutions
    for lambdas in product([0,1], repeat=num_free):
        u = particular + sum(l * v for l, v in zip(lambdas, sol))
        # Reconstruct x and y
        x_full = x_known_vec[:]
        y_full = y_known_vec[:]
        for i, idx in enumerate(unknown_x_indices):
            x_full[idx] = int(u[i])
        for i, idx in enumerate(unknown_y_indices):
            y_full[idx] = int(u[len(unknown_x_indices) + i])
        if sum(x_full) == weight and sum(y_full) == weight:
            return x_full, y_full  # Found a valid solution

    print("No solution found with the required weights.")
    return None


In [15]:
# Take as input two public keys whose private keys are just a shift from one another and the weight of the secret keys, and outputs the shift
def find_shift(h, h1, s, s1, w=w):
    h_sum = h + h1
    if weight(h_sum) % 2 == 0:
        h_sum += ones
    h_inv = h_sum.inverse()
    hs = h_inv * s
    hs1 = h_inv * s1
    for i in range(n):
        if weight(hs + hs1 * xbar^-i)==w:
            return i

    return None

In [16]:
# Crackable if the private keys are just a shift from one another

# attack
h1 = Rq.random_element()
j = randint(0, 1000) # it can go to n-1, but has a risk of taking some minutes to find the shift
s1 = sk[0]*xbar^j + h1 * sk[1]*xbar^j
shift = find_shift(h,h1,s,s1)
y = linsolve(h+h1,s+s1*xbar^-shift)

print('Obtained the secret key:', y==sk[1])

Obtained the secret key: True


In [17]:
# If h has extremely low (or high) weight, <=2 (=>n-2), then it is possible to decrypt the message even without the secret key

# set up
i = randint(0, n-1)
j = randint(0, n-1)
h1 = xbar^i + xbar^j #+ones
s1 = sk[0] + h1 * sk[1]
u1, v1 = hqc_encrypt(h1, s1, msg)

# attack
at_msg = decode(v1)
print('Otained the message:', at_msg==msg)

Otained the message: True


In [18]:
# Probabily of this working, and some information about the weight of the errors
acc=0
err=[]
ors=[]
for _ in range(1000):
    i = randint(0, n-1)
    j = randint(0, n-1)
    h1 = xbar^i + xbar^j
    s1 = sk[0] + h1 * sk[1]
    u1, v1 = hqc_encrypt(h1, s1, msg)
    if decode(v1)==msg:
        acc+=1
        err+=[weight(m+v1)]
    else:
        ors+=[weight(m+v1)]
print(acc/1000)
print(max(err),sum(err)/acc.n())
print(min(ors),sum(ors)/(1000-acc).n())

93/100
7397 7219.54838709677
7193 7324.45714285714


In [19]:
#functions as described in the paper

def compute_Z_explicit(S, sigma):
    # S: list of syndromes S1 ... S_{2t}
    # sigma: error locator polynomial σ(x), degree t

    l = sigma.degree()
    coeffs = [1]

    for i in range(l):
        term = S[i] + sigma[i+1]
        for j in range(i):
            if j < l and (i - j) < len(S): 
                term += sigma[j+1] * S[i - 1 - j]
        coeffs.append(term)

    return PR(coeffs)

def compute_error_values_explicit(Z, positions):
    e = [F(0)] * n_rs
    betas = [a^j for j in positions]
    
    for idx, pos in enumerate(positions):
        beta_inv = a^(-pos)
        numerator = Z(beta_inv)
        denominator = F(1)
        
        for j, beta in enumerate(betas):
            if j != idx:
                denominator *= (F(1) + beta * beta_inv)
        
        e[pos] = numerator / denominator
    
    return e